In [2]:
import instructor
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv

load_dotenv()

# Enables `response_model`
client = instructor.patch(OpenAI())

In [3]:


class UserDetail(BaseModel):
    name: str
    age: int

user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Brian is 46 years old"},
    ]
)

assert isinstance(user, UserDetail)
assert user.name == "Brian"
assert user.age == 46

json_str = user.model_dump_json()
print(json_str)

{"name":"Brian","age":46}
